In [3]:
import numpy as np
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
import random

E:\anaconda\envs\deepmimo\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
Nr =4;Nt=4
channel = torch.from_numpy(np.load('noisechannel.npy',allow_pickle=True))
imagechannel = channel.view(18100,Nr**2,Nt**2)
np.save("./imagechannel.npy",imagechannel)
print(imagechannel.shape)
#print(imagechannel)

torch.Size([18100, 16, 16])


In [13]:
#Gaussian blur
noiseamp = 10**-7
gnoise = noiseamp*(torch.randn(18100,4,4)+torch.randn(18100,4,4)*1j)
gbchannel = imagechannel + gnoise

In [5]:
#Rotate
r90channel = torch.zeros(size = (18100,Nr**2,Nt**2),dtype = torch.cfloat)
r180channel = torch.zeros(size = (18100,Nr**2,Nt**2),dtype = torch.cfloat)
r270channel = torch.zeros(size = (18100,Nr**2,Nt**2),dtype = torch.cfloat)
for counter in range(0,18099):
    rochannel = imagechannel[counter,:,:]
    r90 = np.rot90(rochannel, 1).copy()
    r180 = np.rot90(rochannel, 2).copy()
    r270 = np.rot90(rochannel, 3).copy()
    r90channel[counter,:,:] = torch.from_numpy(r90)
    r180channel[counter,:,:] = torch.from_numpy(r180)
    r270channel[counter,:,:] = torch.from_numpy(r270)

RuntimeError: The expanded size of the tensor (4) must match the existing size (16) at non-singleton dimension 1.  Target sizes: [4, 4].  Tensor sizes: [16, 16]

In [14]:
#Cut out
Ncutout = 2
cochannel = channel/abs(channel)
for counter in range(0,18099):
    cutoutindex = random.sample(range(0,15),Ncutout)
    cochannel[cutoutindex[0],counter]=0
    cochannel[cutoutindex[1],counter]=0
cutoutchannel = cochannel.view(18100,Nr**2,Nt**2)

In [7]:
# flip
vflip = torch.zeros(18100,Nr**2,Nt**2)
hflip = torch.zeros(18100,Nr**2,Nt**2)
for counter in range(0,18099):
    flipchannel = imagechannel[counter,:,:]
    vflip[counter,:,:] = torch.flip(flipchannel,[0])
    hflip[counter,:,:] = torch.flip(flipchannel,[1])

RuntimeError: The expanded size of the tensor (4) must match the existing size (16) at non-singleton dimension 1.  Target sizes: [4, 4].  Tensor sizes: [16, 16]

In [6]:
# tranpose and conjugate
Tchannel = torch.zeros(18100,4,4)
Cchannel = torch.zeros(18100,4,4)
Hchannel = torch.zeros(18100,4,4)
for counter in range(0,18099):
    Tchannel[counter,:,:] = imagechannel[counter,:,:].T
    Cchannel[counter,:,:] = imagechannel[counter,:,:].conj()
    Hchannel[counter,:,:] = (imagechannel[counter,:,:].T).conj()

RuntimeError: The expanded size of the tensor (4) must match the existing size (16) at non-singleton dimension 1.  Target sizes: [4, 4].  Tensor sizes: [16, 16]

In [15]:
print(cutoutchannel)

tensor([[[-0.9838+0.1791j,  0.0000+0.0000j,  0.9218-0.3878j,  ...,
           0.1760-0.9844j, -0.9241+0.3821j,  0.7125+0.7017j],
         [ 0.5817+0.8134j,  0.0000+0.0000j, -0.9620+0.2730j,  ...,
          -0.9366+0.3504j,  0.9504+0.3110j, -0.7004-0.7138j],
         [-0.0281+0.9996j, -0.9983+0.0580j,  0.9988+0.0485j,  ...,
          -0.7583-0.6519j,  0.1907-0.9816j, -0.1744+0.9847j],
         ...,
         [ 0.5680+0.8230j,  0.2574-0.9663j, -0.0300-0.9996j,  ...,
           0.4862-0.8739j, -0.2049+0.9788j, -1.0000+0.0078j],
         [ 0.0000+0.0000j, -0.9855+0.1697j,  0.0000+0.0000j,  ...,
          -0.9127-0.4086j,  0.9704-0.2413j,  0.0000+0.0000j],
         [ 0.0000+0.0000j,  0.9794-0.2020j, -0.7727+0.6348j,  ...,
          -0.1902+0.9818j, -0.2084-0.9780j,  0.1653-0.9862j]],

        [[ 0.4309+0.9024j,  0.7317-0.6816j, -0.8661-0.4998j,  ...,
          -0.6325-0.7745j,  0.0000+0.0000j,  0.8139-0.5810j],
         [-0.9985+0.0546j,  0.8780+0.4787j, -0.3506-0.9365j,  ...,
          -0.9